In [4]:
import os
from getpass import getpass
import json
import random
import textwrap
from openai import OpenAI

client = OpenAI(api_key="KEY")

In [24]:
# ---------- 템플릿(형식별) ----------

# V4: '보기'도 반드시 입력된 문장을 활용하도록 수정
FILL_IN_BLANK_TMPL = """\
[ROLE] 너는 '외국어로서의 한국어' 교재 편집자다. 반드시 JSON만 출력한다.

[GOAL]
- [INPUT_SENTENCES]에 주어진 문장 중 하나를 활용하여 빈칸 채우기 문제 1개를 만든다.

[INPUT_SENTENCES]
{sentences_bullets}

[INSTRUCTIONS]
- `stem_with_blank`: [INPUT_SENTENCES]의 문장 중 하나를 선택하여, 타깃 문법 부분을 빈칸( ___ )으로 바꾼다.
- `example`: **문제에 사용하지 않은 다른 입력 문장 하나를 골라** 동일한 형식의 보기 문항을 생성한다.
- `options`: 정답 1개 + 오답 3개(총 4개). 오답은 학습자가 흔히 저지르는 문법적 오류를 포함하여 구성한다.
- `instruction`: "다음 문장의 빈칸에 들어갈 가장 알맞은 것을 고르십시오." 와 같은 명확한 지시문을 작성한다.

[OUTPUT_JSON_SCHEMA_EXAMPLE]
{{
  "schema_id": "{schema_id}",
  "format": "fill_in_blank",
  "input": {{
    "instruction": "다음 문장의 빈칸에 들어갈 가장 알맞은 것을 고르십시오.",
    "example": {{
        "stem": "나는 점심을 ___ TV를 봤어요.",
        "answer": "먹으면서"
    }},
    "stem_with_blank": "저는 음악을 ___ 공부합니다.",
    "options": ["들으면서", "들었고", "듣지만", "들으려고"]
  }},
  "answer": {{
    "completed_sentence": "저는 음악을 들으면서 공부합니다."
  }},
  "rationale": "두 가지 행동을 동시에 함을 나타내는 '-으면서'가 자연스럽습니다."
}}
"""

# V4: '보기'도 반드시 입력된 문장을 활용하도록 수정
MATCH_AND_CONNECT_TMPL = """\
[ROLE] 너는 '외국어로서의 한국어' 교재 편집자다. 반드시 JSON만 출력한다.

[GOAL]
- 입력된 문장들을 분해하고 재조합하여, 문장 연결하기 문제를 생성한다.

[INPUT_SENTENCES]
{sentences_bullets}

[INSTRUCTIONS]
- `instruction`: "다음 문장을 연결하여 <보기>와 같이 하나의 문장을 만드십시오." 와 같은 지시문을 작성한다.
- `clause_set_A`, `clause_set_B`: 입력된 문장들에서 3~4개를 골라 각각 앞부분과 뒷부분으로 분해하고, 순서를 섞어서 배치한다.
- `example`: **문제에 사용되지 않은 다른 입력 문장 하나를 골라** 분해하여 <보기>를 만든다.
- `answer`: `clause_set_A`의 각 항목에 맞는 `clause_set_B`의 항목을 연결하여 만든 완성 문장들의 배열.

[OUTPUT_JSON_SCHEMA_EXAMPLE]
{{
    "schema_id": "{schema_id}",
    "format": "match_and_connect",
    "input": {{
        "instruction": "다음 문장을 연결하여 <보기>와 같이 하나의 문장을 만드십시오.",
        "example": {{ "clause_A": "아버지는 운동을 하다", "clause_B": "건강을 챙기다", "connected": "아버지는 운동을 하면서 건강을 챙깁니다." }},
        "clause_set_A": ["저는 음악을 듣다", "그녀는 친구와 이야기를 나누다", "우리는 여행 계획을 세우다"],
        "clause_set_B": ["즐거운 시간을 보내다", "공부하다", "웃고 있다"]
    }},
    "answer": {{ "connected_sentences": ["저는 음악을 들으면서 공부합니다.", "그녀는 친구와 이야기를 나누면서 웃고 있습니다.", "우리는 여행 계획을 세우면서 즐거운 시간을 보냈습니다."] }},
    "rationale": "조건을 나타내는 '-으면'을 사용하여 앞선 절과 뒷선 절을 자연스럽게 연결할 수 있습니다."
}}
"""

# V4: '보기'도 반드시 입력된 문장을 활용하도록 수정
SENTENCE_CONNECTION_TMPL = """\
[ROLE] 한국어 문장 연결 문제 출제자. JSON만 출력.

[GOAL]
- 입력된 문장 하나를 두 개의 절로 분해하여 문장 연결 문제를 생성한다.

[INPUT_SENTENCES]
{sentences_bullets}

[INSTRUCTIONS]
- `instruction`: "다음 두 문장을 <보기>와 같이 목표 문법을 사용하여 한 문장으로 만드십시오." 와 같이 지시문을 작성한다.
- `input`: [INPUT_SENTENCES]의 문장 중 하나를 선택하여 두 개의 독립된 문장(`clause_A`, `clause_B`)으로 분해한다.
- `answer`: 분해되기 전의 원본 문장을 `connected_sentence` 값으로 설정한다.
- `example`: **문제에 사용되지 않은 다른 입력 문장 하나를 골라** 분해하여 <보기>를 만든다.

[OUTPUT_JSON_SCHEMA_EXAMPLE]
{{
  "schema_id": "{schema_id}",
  "format": "sentence_connection",
  "input": {{
    "instruction": "다음 두 문장을 <보기>와 같이 '-(으)면서'를 사용하여 한 문장으로 만드십시오.",
    "example": {{ "clause_A": "나는 점심을 먹습니다.", "clause_B": "TV를 봅니다.", "connected": "나는 점심을 먹으면서 TV를 봅니다." }},
    "clause_A": "저는 음악을 듣습니다.",
    "clause_B": "저는 공부를 합니다."
  }},
  "answer": {{ "connected_sentence": "저는 음악을 들으면서 공부합니다." }},
  "rationale": "두 가지 행동이 동시에 일어남을 나타낼 때 동사 어간에 '-으면서'를 붙여 연결할 수 있습니다."
}}
"""

# V4: '보기'도 반드시 입력된 문장을 활용하도록 수정
SENTENCE_CREATION_TMPL = """\
[ROLE] 한국어 문장 생성 문제 출제자. JSON만 출력.

[GOAL]
- 입력된 문장에서 핵심 표현을 추출하여 문장 생성 문제를 만든다.

[INPUT_SENTENCES]
{sentences_bullets}

[INSTRUCTIONS]
- `cues`: [INPUT_SENTENCES]의 문장 중 하나에서 핵심이 되는 표현 2~4개를 추출하여 조합할 요소로 제시한다.
- `created_sentence`: `cues`가 추출된 원본 문장을 정답으로 설정한다.
- `example`: **문제에 사용되지 않은 다른 입력 문장 하나에서** 핵심 표현을 추출하여 <보기>를 만든다.
- `instruction`: "<보기>와 같이 주어진 표현을 사용하여 문장을 완성하십시오." 같은 지시문을 작성한다.

[OUTPUT_JSON_SCHEMA_EXAMPLE]
{{
  "schema_id": "{schema_id}",
  "format": "sentence_creation",
  "input": {{
    "instruction": "<보기>와 같이 주어진 표현을 사용하여 '-(으)면서' 문법으로 문장을 완성하십시오.",
    "example": {{ "cues": ["점심을 먹다", "TV를 보다"], "answer": "점심을 먹으면서 TV를 봅니다." }},
    "cues": ["음악을 듣다", "공부하다"]
  }},
  "answer": {{ "created_sentence": "음악을 들으면서 공부합니다." }},
  "rationale": "핵심 표현들을 '-으면서' 문법을 사용하여 자연스러운 문장으로 만들 수 있습니다."
}}
"""

# (Choice Completion, Dialogue Completion은 구조상 보기 생성이 자유로워 이전 버전을 유지합니다.)
CHOICE_COMPLETION_TMPL = """\
[ROLE] 한국어 문제 출제자. JSON만 출력.

[GOAL]
- 목표 문법: {target_grammar}
- 레벨: {level}
- 제시문(prompt)에 맞는 선택지로 문장을 완성하는 문제 1개 생성.

[INPUT_SENTENCES]
{sentences_bullets}

[INSTRUCTIONS]
- prompt: 간단한 상황/명제 제시문 1개.
- options: 4개(정답 1, 오답 3). 타깃 문법 사용 여부/형태로 갈리게 설계.
- completed_sentence: prompt + 선택지 중 정답을 결합한 완성 문장.

[OUTPUT_JSON_SCHEMA_EXAMPLE]
{{
  "schema_id": "{schema_id}",
  "format": "choice_completion",
  "input": {{
    "prompt": "제시문",
    "options": ["선택지1","선택지2","선택지3","선택지4"]
  }},
  "answer": {{
    "completed_sentence": "완성 문장"
  }},
  "rationale": "근거 한두 문장"
}}
"""

DIALOGUE_COMPLETION_TMPL = """\
[ROLE] 한국어 대화 완성 문제 출제자. JSON만 출력.

[GOAL]
- 목표 문법: {target_grammar}
- 레벨: {level}
- 대화(turn 3개 내외)에서 1곳을 빈칸으로 두고 자연스럽게 채우게 한다.

[INPUT_SENTENCES]
{sentences_bullets}

[INSTRUCTIONS]
- dialogue_with_missing_turns: A/B 대화 배열. 한 턴은 "___" 로 빈칸 표기.
- completed_dialogue: 빈칸을 채운 최종 대화 배열.
- 타깃 문법은 최소 1회 자연스럽게 등장.

[OUTPUT_JSON_SCHEMA_EXAMPLE]
{{
  "schema_id": "{schema_id}",
  "format": "dialogue_completion",
  "input": {{
    "dialogue_with_missing_turns": [
      {{"speaker":"A","text":"..."}},
      {{"speaker":"B","text":"___"}},
      {{"speaker":"A","text":"..."}}
    ]
  }},
  "answer": {{
    "completed_dialogue": [
      {{"speaker":"A","text":"..."}},
      {{"speaker":"B","text":"채워진 문장"}},
      {{"speaker":"A","text":"..."}}
    ]
  }},
  "rationale": "문맥 상의 연결 근거"
}}
"""

# ---------- 템플릿 매핑 (최종 버전) ----------
TMPLS = {
    "fill_in_blank": FILL_IN_BLANK_TMPL,
    "match_and_connect": MATCH_AND_CONNECT_TMPL,
    "dialogue_completion": DIALOGUE_COMPLETION_TMPL,
    "choice_completion": CHOICE_COMPLETION_TMPL, # choice_completion은 보기 필드가 없으므로 이전 버전 사용
    "sentence_connection": SENTENCE_CONNECTION_TMPL,
    "sentence_creation": SENTENCE_CREATION_TMPL,
}

In [25]:
# ---------- 랜덤 선택 ----------
def pick_random_schema(schema_list, seed=None):
    rng = random.Random(seed)
    return rng.choice(schema_list)

def bullets(sentences):
    return "\n".join(f"- {s}" for s in sentences)

# ---------- 프롬프트 렌더러 ----------
def render_prompt(schema_def, target_grammar, level, sentences):
    fmt = schema_def["format"]
    tmpl = TMPLS[fmt]
    return textwrap.dedent(tmpl).format(
        target_grammar=target_grammar,
        level=level,
        sentences_bullets=bullets(sentences),
        schema_id=schema_def["id"]
    )

In [26]:
def call_llm(prompt: str) -> str:
    """
    OpenAI의 gpt-4o-mini 모델을 호출하여 프롬프트에 대한 응답을 JSON 형식으로 반환합니다.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            response_format={"type": "json_object"} # JSON 모드 활성화
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        # 오류 발생 시 스키마에 맞는 빈 JSON을 반환하거나, 오류 처리를 할 수 있습니다.
        return json.dumps({"error": str(e)})

In [27]:
def generate_one_item(payload):
    """
    전체 문제 생성 파이프라인을 실행합니다.
    입력 payload에서 유효한 문장만 필터링하여 사용합니다.
    """
    seed = payload.get("seed", 0)
    schema_list = payload["schema"]
    target_grammar = payload["target_grammar"]
    level = payload["level"]
    
    # critique_summary에서 is_valid가 True인 문장만 추출
    critique_summary = payload.get("critique_summary", [])
    valid_sentences = [
        item["sentence"] for item in critique_summary if item.get("is_valid", False)
    ]
    
    if not valid_sentences:
        return {"error": "No valid sentences provided to generate a question."}

    chosen = pick_random_schema(schema_list, seed=seed)
    prompt = render_prompt(chosen, target_grammar, level, valid_sentences)
    
    print("="*20, "Generated Prompt to LLM", "="*20)
    print(prompt)
    print("="*60)
    
    raw_json_output = call_llm(prompt)

    try:
        parsed_output = json.loads(raw_json_output)
    except json.JSONDecodeError:
        return {"error": "Failed to decode JSON from LLM response.", "raw_output": raw_json_output}

    return {
        "version": "kfl-ai.v2-gpt4omini",
        "target_grammar": target_grammar,
        "level": level,
        "items": [parsed_output],
        "meta": {"seed": seed, "schema_chosen": chosen["id"]}
    }

In [28]:
# ---------- 문제 생성을 위한 입력 데이터 정의 ----------
payload = {
    "level": 3,
    "target_grammar": "-(으)면서",
    "seed": 201, # 결과를 재현하고 싶을 때 시드 값을 고정
    "schema": [ # 생성하고 싶은 문제 유형들을 리스트로 정의
        {"id": "Q1", "format": "fill_in_blank"},
        {"id": "Q2", "format": "choice_completion"},
        {"id": "Q3", "format": "dialogue_completion"},
        {"id": "Q4", "format": "sentence_connection"},
        {"id": "Q5", "format": "sentence_creation"}
    ],
    "critique_summary": [
        {"sentence": "저는 음악을 들으면서 공부합니다.", "is_valid": True, "reason": "OK"},
        {"sentence": "그녀는 친구와 이야기를 나누면서 웃고 있습니다.", "is_valid": True, "reason": "OK"},
        {"sentence": "나는 점심을 먹으면서 TV를 봤어요.", "is_valid": True, "reason": "주어 일치"}, # 예시 데이터의 is_valid를 수정하여 유효한 문장으로 간주
        {"sentence": "아버지는 운동을 하면서 건강을 챙깁니다.", "is_valid": True, "reason": "OK"},
        {"sentence": "학생들은 발표를 준비하면서 긴장하고 있어요.", "is_valid": True, "reason": "OK"},
        {"sentence": "우리는 여행 계획을 세우면서 즐거운 시간을 보냈습니다.", "is_valid": True, "reason": "OK"}
    ]
}

# ---------- 문제 생성 실행 ----------
generated_question = generate_one_item(payload)


# ---------- 결과 출력 ----------
print("\n\n", "="*20, "Final Output JSON", "="*20)
print(json.dumps(generated_question, indent=2, ensure_ascii=False))

==================== Generated Prompt to LLM ====================
[ROLE] 너는 '외국어로서의 한국어' 교재 편집자다. 반드시 JSON만 출력한다.

[GOAL]
- [INPUT_SENTENCES]에 주어진 문장 중 하나를 활용하여 빈칸 채우기 문제 1개를 만든다.

[INPUT_SENTENCES]
- 저는 음악을 들으면서 공부합니다.
- 그녀는 친구와 이야기를 나누면서 웃고 있습니다.
- 나는 점심을 먹으면서 TV를 봤어요.
- 아버지는 운동을 하면서 건강을 챙깁니다.
- 학생들은 발표를 준비하면서 긴장하고 있어요.
- 우리는 여행 계획을 세우면서 즐거운 시간을 보냈습니다.

[INSTRUCTIONS]
- `stem_with_blank`: [INPUT_SENTENCES]의 문장 중 하나를 선택하여, 타깃 문법 부분을 빈칸( ___ )으로 바꾼다.
- `example`: **문제에 사용하지 않은 다른 입력 문장 하나를 골라** 동일한 형식의 보기 문항을 생성한다.
- `options`: 정답 1개 + 오답 3개(총 4개). 오답은 학습자가 흔히 저지르는 문법적 오류를 포함하여 구성한다.
- `instruction`: "다음 문장의 빈칸에 들어갈 가장 알맞은 것을 고르십시오." 와 같은 명확한 지시문을 작성한다.

[OUTPUT_JSON_SCHEMA_EXAMPLE]
{
  "schema_id": "Q1",
  "format": "fill_in_blank",
  "input": {
    "instruction": "다음 문장의 빈칸에 들어갈 가장 알맞은 것을 고르십시오.",
    "example": {
        "stem": "나는 점심을 ___ TV를 봤어요.",
        "answer": "먹으면서"
    },
    "stem_with_blank": "저는 음악을 ___ 공부합니다.",
    "options": ["들으면서", "들었고", "듣지만", "들으려고"]
  },
  "ans